# What's cooking?

SVC

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
train_df = pd.read_json('./train.json')
test_df = pd.read_json('./test.json')

train_df.sample(5)
# coletando ingredientes:
ingredients = train_df.ingredients.str.join(' ')
# ingredients
# removendo caracteres especiais, stemming e lemmatizing:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]

train = train_df.copy()
train['X'] = train_df.ingredients.str.join(' ')
train['X'].apply(lemmatize_text)

train.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,cuisine,ingredients,X
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
vectorized_ingredients = count_vectorizer.fit_transform(train.X)
print('20 features: '+ str(count_vectorizer.get_feature_names()[0:20]))
print('Total features: '+ str(len(count_vectorizer.get_feature_names())))
print(vectorized_ingredients.toarray()[0:10])

vectorized_ingredients_df = pd.DataFrame(columns = count_vectorizer.get_feature_names(), data = vectorized_ingredients.toarray())
vectorized_ingredients_df['num_ingr'] = vectorized_ingredients_df.sum(axis = 1)
vectorized_ingredients_df.head()

20 features: ['00', '10', '100', '14', '15', '25', '33', '40', '43', '95', '96', 'abalone', 'abbamele', 'absinthe', 'abura', 'acai', 'accent', 'accompaniment', 'achiote', 'acid']
Total features: 3010
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


,00,10,100,14,15,25,33,40,43,95,...,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices,num_ingr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33


In [14]:
# encode (label) targets:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cuisine = label_encoder.fit_transform(train_df['cuisine'].values)
cuisine

array([ 6, 16,  4, ...,  8,  3, 13])

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
y = cuisine.copy()
X = vectorized_ingredients_df.values
num_selected = 1000
X_count_selected = SelectKBest(chi2, k=num_selected).fit_transform(X, y)
X_count_selected.shape

(39774, 1000)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True)

tfidf_vectorized_ingredients = tfidf_vectorizer.fit_transform(train.X)
print('20 features: '+ str(tfidf_vectorizer.get_feature_names()[0:20]))
print('Total features: '+ str(len(tfidf_vectorizer.get_feature_names())))
print(tfidf_vectorized_ingredients.toarray()[0:10])

tfidf_vectorized_ingredients_df = pd.DataFrame(columns = tfidf_vectorizer.get_feature_names(), data = tfidf_vectorized_ingredients.toarray())
tfidf_vectorized_ingredients_df['sum'] = tfidf_vectorized_ingredients_df.sum(axis = 1)
tfidf_vectorized_ingredients_df.head()


20 features: ['00', '10', '100', '14', '15', '25', '33', '40', '43', '95', '96', 'abalone', 'abbamele', 'absinthe', 'abura', 'acai', 'accent', 'accompaniment', 'achiote', 'acid']
Total features: 3010
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,00,10,100,14,15,25,33,40,43,95,...,za,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
y = cuisine.copy()
X = tfidf_vectorized_ingredients_df.values.copy()
X_tfidf_selected = SelectKBest(chi2, k=num_selected).fit_transform(X, y)
X_tfidf_selected.shape

(39774, 1000)

In [19]:
# join data
y = cuisine.copy()
print(X_count_selected.shape)
print(X_tfidf_selected.shape)
X = np.concatenate((X_count_selected.copy(),X_tfidf_selected.copy()), axis=1)
X.shape

(39774, 1000)
(39774, 1000)


(39774, 2000)

In [29]:
# https://stackoverflow.com/questions/53940258/svc-classifier-taking-too-much-time-for-training
from sklearn.model_selection import train_test_split
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y, test_size=0.6, random_state=42)
print(X_train3.shape) #0.85 => 4min #0.6 => 17min
print(y_train3.shape)

(15909, 2000)
(15909,)


In [30]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

classifier = SVC(kernel='rbf',random_state=0, probability=False, cache_size = 4000)
model = OneVsRestClassifier(classifier, n_jobs=-1)
model.fit(X_train3, y_train3)

OneVsRestClassifier(estimator=SVC(cache_size=4000, random_state=0), n_jobs=-1)

In [31]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [33]:
_, X_test4, _, y_test4 = train_test_split(X_test3, y_test3, test_size=0.1, random_state=42)
print(X_test4.shape)
print(y_test4.shape)
y_pred = model.predict(X_test4)
print('Accuracy: '+str(accuracy_score(y_test4, y_pred)))
print(classification_report(y_test4, y_pred, target_names=train.cuisine.unique()))

(2387, 2000)
(2387,)
Accuracy: 0.6761625471302891
              precision    recall  f1-score   support

       greek       0.55      0.19      0.29        31
 southern_us       0.32      0.12      0.17        50
    filipino       0.66      0.60      0.63        85
      indian       0.65      0.84      0.73       172
    jamaican       0.70      0.33      0.45        42
     spanish       0.72      0.31      0.43       158
     italian       0.71      0.32      0.44        68
     mexican       0.73      0.85      0.78       192
     chinese       0.77      0.23      0.35        44
     british       0.64      0.91      0.75       462
        thai       0.62      0.52      0.57        29
  vietnamese       0.78      0.49      0.61        87
cajun_creole       0.81      0.46      0.58        57
   brazilian       0.73      0.95      0.83       366
      french       0.77      0.50      0.61        48
    japanese       0.50      0.19      0.28        21
       irish       0.66      0.